In [22]:
import pandas as pd

dataset = pd.read_excel('Raisin_Dataset.xlsx')
X = dataset.drop(columns=['Class'])
print(dataset.columns)


Index(['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity',
       'ConvexArea', 'Extent', 'Perimeter', 'Class'],
      dtype='object')


In [ ]:
from sklearn.impute import KNNImputer
missing_values = dataset.isnull().sum()
knn_imputer = KNNImputer(n_neighbors=5)
dataset_imputed = knn_imputer.fit_transform(dataset)
dataset_imputed = pd.DataFrame(dataset_imputed, columns=dataset.columns)
dataset = dataset_imputed
print(dataset)

In [17]:
from sklearn.feature_selection import mutual_info_classif

y = dataset['Class']

mutual_info_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
mutual_info_df = pd.DataFrame({'Feature': X.columns, 'Mutual Info Score': mutual_info_scores})
mutual_info_df_sorted = mutual_info_df.sort_values(by='Mutual Info Score', ascending=False)
sorted_feature_order = mutual_info_df_sorted['Feature'].tolist()

print("Sorted Feature Order:", sorted_feature_order)


Sorted Feature Order: ['Perimeter', 'MajorAxisLength', 'ConvexArea', 'Area', 'MinorAxisLength', 'Eccentricity', 'Extent']


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

y_prediction = classifier.predict(X_test)

error_rate = 1-accuracy_score(y_test, y_prediction)
print("Error Rate = ",error_rate)



Error Rate =  0.21111111111111114


In [23]:
X = dataset.drop(columns=['Class', 'Extent'])
y = dataset['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

y_prediction = classifier.predict(X_test)

error_rate = 1-accuracy_score(y_test, y_prediction)
print("Error Rate = ",error_rate)

Error Rate =  0.18888888888888888


In [24]:
X = dataset.drop(columns=['Class', 'Extent', 'Eccentricity'])
y = dataset['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

y_prediction = classifier.predict(X_test)

error_rate = 1-accuracy_score(y_test, y_prediction)
print("Error Rate = ",error_rate)

Error Rate =  0.21666666666666667


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
X = dataset.drop(columns=['Class'])
y = dataset['Class'] 
prev_error_rate = float('inf')
i = 1
while True:
    
    excluded_features = ['Eccentricity', 'Extent']  
    
    
    excluded_features.extend(['f{}'.format(i), 'f{}'.format(i-1)])
    
    X_filtered = X.drop(columns=excluded_features)
    
    clf = DecisionTreeClassifier(random_state=42)
    
    param_grid = {'max_depth': [None, 10, 20, 30],
                  'min_samples_split': [2, 5, 10]}
    
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, scoring='accuracy')
    
    cv_scores = cross_val_score(grid_search, X_filtered, y, cv=10)
    
    cv_accuracy = cv_scores.mean()

    error_rate_i = 1 - cv_accuracy
    
    print(f"Error Rate (ED-{i}): {error_rate_i}")
    if error_rate_i >= prev_error_rate:
        break
    prev_error_rate = error_rate_i
    i += 1


KeyError: "['f1', 'f0'] not found in axis"